In [1]:
import numpy as np

def complementary_map(kraus_ops):
    """
    Compute the Kraus operators for the complementary map given the input Kraus operators.
    
    :param kraus_ops: List of Kraus operators (numpy arrays)
    :return: Kraus operators for the complementary map (as a list of numpy arrays)
    """
    # Number of Kraus operators and size of each operator
    num_kraus = len(kraus_ops)
    op_dim = kraus_ops[0].shape[0]  # Dimension of the matrix (assuming square)

    # Initialize the complementary Kraus operators as empty lists
    comp_kraus_ops = []

    # Construct the complementary map by rows
    for row in range(op_dim):
        # Collect all the rows corresponding to 'row' in all Kraus operators
        comp_kraus_op = np.vstack([kraus_ops[i][row, :] for i in range(num_kraus)])
        comp_kraus_ops.append(comp_kraus_op)

    return comp_kraus_ops

# Example usage

# Define the Kraus operators for Phi
kraus_ops_Phi = [
    np.array([[1, 0], [0, 1]]) / np.sqrt(2),
    np.array([[0, 1], [1, 0]]) / np.sqrt(2),
    np.array([[0, -1j], [1j, 0]]) / np.sqrt(2),
    np.array([[1, 0], [0, -1]]) / np.sqrt(2)
]

# Compute the complementary map's Kraus operators
comp_kraus_ops = complementary_map(kraus_ops_Phi)

# Output the result
print("Kraus operators for the complementary map:")
for i, op in enumerate(comp_kraus_ops):
    print(f"Kraus operator {i + 1}:\n{op}\n")


Kraus operators for the complementary map:
Kraus operator 1:
[[ 0.70710678+0.j          0.        +0.j        ]
 [ 0.        +0.j          0.70710678+0.j        ]
 [ 0.        +0.j         -0.        -0.70710678j]
 [ 0.70710678+0.j          0.        +0.j        ]]

Kraus operator 2:
[[ 0.        +0.j          0.70710678+0.j        ]
 [ 0.70710678+0.j          0.        +0.j        ]
 [ 0.        +0.70710678j  0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.j        ]]

